# Prompt engineering

OpenAI promp engineering for using API requests to extract features from the news collected.

Using openai=2.1.0

Using structured output with BaseModel


In [ ]:
from __future__ import annotations
from typing import List, Optional, Literal
from pydantic import BaseModel, Field
import pandas as pd
pd.options.display.max_columns = None
from pandas import json_normalize

from openai import OpenAI

project_path = "C:/Users/santt/Desktop/DataMining_UBA/5-text_mining/nlp_dmuba/"

### Loading dataset

In [ ]:
dataset_file_path = project_path+"1-Scraping/dataset_consolidado/df.parquet" 
df = pd.read_parquet(dataset_file_path)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# taking a sample of 50 news
sample = 50

df_sample = df.sample(sample, random_state=42).reset_index(drop=True)

In [ ]:
df_sample

### OpenAI Requests

Initiating OpenAI client

In [ ]:
try:
    with open(project_path+'.secrets/openai_api_key.txt', 'r') as file:
        key = file.read()
except FileNotFoundError:
    print("Error: The file 'openai_api_key.txt' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

client = OpenAI(api_key = key)

Structure request

In [ ]:
# ---------------------------
# DEFINICIONES DE CONVENCIÓN
# ---------------------------
# Escalas:
# - Señales y sentimientos: [-1..1]  (dirección y magnitud a la vez; 0 = neutro)
# - Confianza y calidad:    [0..1]
#
# Señales de mercado (signo):
# - merval: + sube índice; - baja índice
# - fx_usdars: + se deprecia ARS (sube USD/ARS); - se aprecia ARS (baja USD/ARS)
# - tasa_bcra: + sube tasa de política; - baja tasa
# - bonos_soberanos: + sube precio (mejora riesgo / bajan rendimientos); - baja precio
# - actividad_economica: + mayor actividad; - menor actividad
#
# Reglas de simplicidad (para gpt-5-nano):
# - Si el artículo no da evidencia clara → devolver 0.0 (neutro) y/o "unknown".
# - horizonte_dias solo si aparece explícito (fechas, “en X días/meses”, “desde hoy”).
# - Listas sin duplicados; tickers en MAYÚSCULAS; strings en minúsculas salvo nombres propios.
# - Redondeo de floats a 2 decimales recomendado en el post-procesamiento.

class SenalesMercado(BaseModel):
    merval: float = Field(description="[-1..1] Sesgo esperado para el índice Merval (↑:+, ↓:-).")
    fx_usdars: float = Field(description="[-1..1] Sesgo para el tipo de cambio USD/ARS (↑:+ deprecia ARS, ↓:- aprecia ARS).")
    tasa_bcra: float = Field(description="[-1..1] Sesgo para la tasa de política del BCRA (↑:+, ↓:-).")
    bonos_soberanos: float = Field(description="[-1..1] Sesgo para el precio de bonos soberanos (↑:+ precio, ↓:- precio).")
    actividad_economica: float = Field(description="[-1..1] Sesgo sobre nivel de actividad (↑:+, ↓:-).")

class Sentimientos(BaseModel):
    valencia_general: float = Field(description="[-1..1] Tono global del artículo sobre economía/mercados.")
    gobernanza: float = Field(description="[-1..1] Tono respecto a gobierno/gobernanza (institucional/político).")
    expectativa_macro_corto: float = Field(description="[-1..1] Expectativa macro a 1–3 meses.")
    expectativa_macro_largo: float = Field(description="[-1..1] Expectativa macro a >6 meses.")
    expectativa_fin_corto: float = Field(description="[-1..1] Condiciones financieras a 1–3 meses (tasas, crédito, riesgo).")
    expectativa_fin_largo: float = Field(description="[-1..1] Condiciones financieras a >6 meses.")

class Tematicas(BaseModel):
    menciona_inflacion: bool = Field(description="Se menciona inflación/precios.")
    menciona_pbi: bool = Field(description="Se menciona PBI/crecimiento.")
    menciona_reservas: bool = Field(description="Se mencionan reservas del BCRA.")
    menciona_embi: bool = Field(description="Se menciona riesgo país/EMBI.")
    menciona_deuda: bool = Field(description="Se menciona deuda pública/privada.")
    menciona_fmi: bool = Field(description="Se menciona FMI/acuerdos.")
    menciona_salarios_paritarias: bool = Field(description="Se mencionan salarios/paritarias.")
    menciona_tipo_cambio: bool = Field(description="Se menciona tipo de cambio/dólar.")
    menciona_confianza_consumidor: bool = Field(description="Se menciona índice/sentimiento de confianza del consumidor.")

class Entidades(BaseModel):
    tipo_actor_principal: Literal[
        "gobierno_nacional","bcra","provincia","municipio",
        "empresa_local","empresa_extranjera",
        "sindicato","poder_judicial","congreso","organismo_internacional",
        "desconocido"
    ] = Field(description="Actor dominante al que refiere la noticia.")
    nombre_actor_principal: Optional[str] = Field(default="unknown", description="Nombre del actor (si es claro).")
    empresas_mencionadas: List[str] = Field(default_factory=list, description="Empresas citadas (nombre legal).")
    tickers_mencionados: List[str] = Field(default_factory=list, description="Tickers (BYMA/ADRs) en MAYÚSCULAS, sin duplicados.")
    sectores_mencionados: List[str] = Field(default_factory=list, description="Sectores/industrias relevantes (ej.: bancos, energía).")

class Evento(BaseModel):
    tipo_evento: Literal[
        "monetario","fiscal","regulatorio","corporativo","externo",
        "sindical_social","judicial","electoral","otro","desconocido"
    ] = Field(description="Categoría simple del hecho principal.")
    shock: Literal["positivo","negativo","mixto","neutro","desconocido"] = Field(description="Signo cualitativo del shock.")
    caracter: Literal["retroactivo","vigente","prospectivo","desconocido"] = Field(description="Temporalidad legal/efectiva.")
    horizonte_dias: Optional[int] = Field(default=None, description="Días hasta impacto si aparece explícito; si no, null.")

class CalidadFuente(BaseModel):
    categoria: Literal["oficial","periodistica","rumor","analisis","desconocido"] = Field(
        description="Tipo de fuente del contenido."
    )
    score: float = Field(description="[0..1] Confiabilidad de la fuente según categoría (regla guía en prompt).")

class NewFeatures(BaseModel):
    # Núcleo
    entidades: Entidades
    evento: Evento
    mercado: SenalesMercado
    sentimientos: Sentimientos
    tematicas: Tematicas

    # Calidad / confianza
    calidad_fuente: CalidadFuente
    confianza: float = Field(description="[0..1] Confianza global de extracción (claridad y evidencia).")

### SYSTEM_PROMPT

SYSTEM_PROMPT = '''
Eres un analista económico-financiero especializado en Argentina.
Objetivo: extraer datos ESTRUCTURADOS de una noticia para modelar el MERVAL.
Reglas de oro (optimizado para modelos pequeños):
- Usa SOLO el texto de la noticia. NO infieras más allá.
- Si la evidencia no es clara: usa 0.0 (neutro), "unknown" o null (horizonte).
- Señales y sentimientos son valores en [-1..1] (dirección y magnitud a la vez).
- Confianza y calidad de fuente en [0..1].
- “horizonte_dias” SOLO si hay mención explícita (“en X días/meses”, “desde hoy”, una fecha clara).
- Tickers en MAYÚSCULAS; listas sin duplicados.
- No cites fuentes externas.

Guías rápidas:
- Calidad de fuente → score por defecto:
  oficial: 0.9, periodistica: 0.7, analisis: 0.6, rumor: 0.3, desconocido: 0.5
- Señales de mercado (signo):
  merval (+ sube índice), fx_usdars (+ se deprecia ARS), tasa_bcra (+ sube tasa),
  bonos_soberanos (+ sube precio), actividad_economica (+ sube actividad).
- Si el artículo es de temática no económica/financiera: deja señales en 0.0, evento=“desconocido”, confianza ≤ 0.3.

Salida: JSON ESTRICTO que valide contra el esquema. No agregues texto extra.
'''
### USER_TEMPLATE (sin pasos “pesados”, directo al punto)

USER_TEMPLATE = '''
Diario: {diario}
Fecha: {fecha}  # formato YYYY-MM-DD
Seccion: {seccion}
Titulo: {titulo}
Contenido: {contenido}

Devuelve SOLO el JSON con el esquema pedido (sin texto adicional).
'''


In [ ]:
model = "gpt-5-mini"

results = []
for _, row in df_sample.iterrows():
    print(f"Processing row {_}/{len(df_sample)}")
    # Manejo de faltantes
    contenido = (row.get("contenido") or "")[:8000]
    prompt = USER_TEMPLATE.format(
        diario=row.get("diario", "unknown"),
        fecha=str(row.get("fecha", "unknown")),
        seccion=row.get("seccion", "unknown"),
        titulo=row.get("titulo", "unknown"),
        contenido=contenido,
        url=row.get("url", "unknown"),
    )

    resp = client.responses.parse(
        model=model,
        input=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
        text_format=NewFeatures,
    )

    parsed = resp.output_parsed
    results.append(parsed.model_dump())

df_features = json_normalize(results, sep="__")
df_sample_features = pd.concat([df_sample.reset_index(drop=True), df_features], axis=1)

Took 21 minutes for 50 news

In [ ]:
df_sample_features

In [ ]:
df_sample_features.info()

In [ ]:
df_sample_features.to_csv(project_path+f"5-LLMs/openai/df_sample{sample}_{model}.csv", index=False)

To decide and improve:

- Avoiding quotes and other kind of feature requirend using more output tokens
- Avoiding reducing too complex features
- Aggregating news by day / by newspaper
